# Validate the annotated datasets

This notebook provides code for validating the evaluation set .txt files and retrieving information about the anthropomorphic components.

1. Make sure that no sentence was annotated with conflicting annotations
2. Make sure that there are no duplicate sentences in a sentence
3. Make sure that the .txt files used to create the evaluation sets are well-formed - i.e, the IDs contain the database prefix (used to locate them in the dataframe) and that each row contains exactly seven tab-separated values.
4. Check that the annotations are correct - e.g. the positive set contains only ['p1','p2','p3'] scores, the negative set contains only ['n1','n2','n3'] scores, and the inconclusive set has only 'inc'.
5. Retrieving the anthropomorphic components
6. Retrieving the AI entity lemmas (i.e. without descriptors and modifiers)

In [144]:
import re
        
def get_sentences_dict(cat,score):   

    sentences_dict = {}
    duplicate_ids = []
    duplicate_sentence_pairs = []
    
    sentences = open(f"../preprocessed_data/evaluation_sentences/{cat}_{score}.txt","r")
    
    for line in sentences.readlines():
        line = line.strip()
        line = line.split("\t")
        if len(line) == 0:
            break
        sent_id = line[0]
        sent_info = line[1:]

        # wellformedness checks
        if len(line) != 7:
            print(f"The row with the ID {sent_id} in {cat}_{score}.txt is not well-formed.")
        id_prefix = sent_id[:6]
        if not re.match(r"^[1-7]{1}_(arx|acl)_", id_prefix):
            print(f"The ID {sent_id} in {cat}_{score} is not well-formed.")
        
        if sent_id not in sentences_dict:
            if sent_info not in sentences_dict.values():
                sentences_dict[sent_id] = sent_info
            else: # the sentence appears twice with different IDs 
                other_id = [key for key in sentences_dict if sentences_dict[key] == sent][0]
                duplicate_sentence = (other_id,sent_id)
                duplicate_sentence_pairs.append(duplicate_sentence) 
        else: # the sentence appears twice with the same ID
            duplicate_ids.append(sent_id)

    return sentences_dict,duplicate_ids,duplicate_sentence_pairs

In [145]:
def check_num_and_duplicates(cat,score,num):

    response = "No duplicate utterances."
    print(f"Checking for duplicate entries in {cat}_{score}.txt...")

    sentences_dict = get_sentences_dict(cat,score)[0] # dict of ids and sentence info
    duplicate_ids = get_sentences_dict(cat,score)[1] # list of duplicate sentences with identical ids
    duplicate_sentence_pairs = get_sentences_dict(cat,score)[2] # list of duplicate sentences with different ids
    
    if len(sentences_dict.keys()) > num:
        print(f"There are more than {num} sentences in {cat}_{score}.txt.")
    elif len(sentences_dict.keys()) < num:
        print(f"There are less than {num} sentences in {cat}_{score}.txt.")

    if duplicate_ids:
        response = f"Resolve duplicates in the {score} set!!!"
        print("The sentences with the following ids appear twice: ",duplicate_ids,
             f" in {cat}_{score}.txt")

    if duplicate_sentence_pairs:
        response = f"Resolve duplicates in the {score} set!!!"
        print("The following ID pairs refer to the same sentence: ",duplicate_sentence_pairs,
             f" in {cat}_{score}.txt")

    return response

def check_annotations(cat,score):

    annotations_dict = {}

    if cat == "noun_phrases":
        annotations = ['p']
    else:
        if score == 'positive':
            annotations = ['p1','p2','p3']
        elif score == 'negative':
            annotations = ['n1','n2','n3']
        elif score == 'inconclusive':
            annotations = ['inc']

    print(f"Checking annotations in {cat}_{score}.txt:")

    sentences_dict = get_sentences_dict(cat,score)[0]
    all_sentences_info = sentences_dict.values()

    for sent_id,sent_info in sentences_dict.items():
        if sent_info[5] not in annotations:
            print(f"Fix incorrect annotation {sent_info[5]} in the sentence with the ID {sent_id}")
        if sent_info[5] not in annotations_dict:
            annotations_dict[sent_info[5]] = 1
        else:
            annotations_dict[sent_info[5]] += 1

    return annotations_dict

def pairwise_conflict_check(cat,score1,score2):

    print(f"Comparing {score1} cases and {score2} cases for the {cat} set...")

    conflicting_annotation = False

    dict1 = get_sentences_dict(cat,score1)[0]
    dict2 = get_sentences_dict(cat,score2)[0]

    for id1,sent in dict1.items():
        if id1 in dict2:
            conflicting_annotation = True
            print(f"The {score1} sentence with the ID ",id1,f" appears in the {score2} set with the same ID")
        elif sent in dict2.values():
            conflicting_annotation = True
            id2 = [key for keys in dict2.keys() if dict2[key] == sent][0]
            print(f"The {score1} sentence with the ID  ",id1,
                  f" appears in the {score2} set with the ID ",id2)

    return conflicting_annotation

def check_conflicting_annotations(cat,case,other_cases):

    response = "No conflicting annotations."

    for other_case in other_cases:

        conflicting_annotations = pairwise_conflict_check(cat,case,other_case)
        if conflicting_annotations:
            check = "Resolve conflicts before proceeding."
            print(f"Conflicting annotations in the {case} and {other_case} sets!!!")

    return response

In [114]:
def get_anthro_components(cat,score):

    anthro_components = {}

    print(f"Retrieving a list of anthropomorphic words in {cat}_{score}.txt:")
    
    sentences_dict = get_sentences_dict(cat,score)[0]
    all_sentences_info = sentences_dict.values()
    
    for sent_info in all_sentences_info:
        if sent_info[4] not in anthro_components:
            anthro_components[sent_info[4]] = 1
        else:
            anthro_components[sent_info[4]] += 1

    return anthro_components

def get_ai_components(cat,score):

    ai_components = {}

    print(f"Retrieving a list of AI entities in {cat}_{score}.txt:")
    
    sentences_dict = get_sentences_dict(cat,score)[0]
    all_sentences_info = sentences_dict.values()
    
    for sent_info in all_sentences_info:
        if sent_info[3] not in ai_components:
            ai_components[sent_info[3]] = 1
        else:
            ai_components[sent_info[3]] += 1

    return ai_components

#### Check sentences for each category

The categories are:
1. verb_subjects - sentences in which the AI entity is the subject of an anthropomorphic verb (nsubj)
2. verb_objects - sentences in which the AI entity is object of an anthropomorphic verb (pobj,dobj)
4. adjective_phrases - sentences in which the AI entity is part of an anthropomorphic adjectival phrase
5. noun_phrases - sentences in which the AI entity is part of an anthropomorphic noun phrase
6. possessives - sentences in which the AI entity is immediately followed by a possessive marker
7. comparisons - sentences in which the AI entity is being compared to humans explicitly

In [156]:
#cases_and_nums = {"positive":50,"negative":50,"inconclusive":20}
cases_and_nums = {"inconclusive":50}
category_is = "comparisons" # bring it to the runway

for case in cases_and_nums:

    # check that the number of utterance matches the expecation, and that the file contains no duplicate sentences
    check1 = check_num_and_duplicates(category_is,case,cases_and_nums[case])
    print(check1,'\n')
    
    # check that the same sentence does not appear twice in two sets of the same category
    # not applicable for noun_phrases (always positive) and comparisons (always inconclusive)
    if category_is != "noun_phrases" and category_is != "comparisons":
        other_cases = [other_case for other_case in cases_and_nums if other_case != case]
        check2 = check_conflicting_annotations(category_is,case,other_cases)
        print(check2, '\n')

    # check that the annotations in a given file are correct (i.e. no negative annotations in the positive set)
    check3 = check_annotations(category_is,case)
    print(check3,'\n')

    # retrieve all of the (non-)anthropomorphic components in a given file - the verb/noun/adjective in suspect
    anthro_components = get_anthro_components(category_is,case)
    print(anthro_components,'\n')

    # retrieve all of the AI components in a given file
    ai_components = get_ai_components(category_is,case)
    print(ai_components,'\n')

Checking for duplicate entries in comparisons_inconclusive.txt...
No duplicate utterances. 

Checking annotations in comparisons_inconclusive.txt:
{'inc': 50} 

Retrieving a list of anthropomorphic words in comparisons_inconclusive.txt:
{'perform similarly to humans': 2, 'score similarly to human samples in terms of personality': 1, 'inference process similarly to humans': 1, 'human-like inferences': 1, 'human-like expression and reasoning abilitiies': 1, 'process visual information in a human-like way': 1, 'respond to conversations in a human-like manner': 1, 'human-like syntactic and semantic knowledge': 1, 'human-like reasoning': 1, 'human-like fluency': 1, 'human-like conversations': 1, 'human-like mistakes': 1, 'human-like responses in dialogue': 1, 'human-like': 1, 'human-like cognitive abilities': 2, 'creative as humans': 1, 'recognize various entities to the same degree as humans': 1, 'converse with each other as well as humans': 1, 'reason casually as humans would': 1, 'human-